## Transportation problem with ramp-up costs

### Model parameters
* $P$: the set of production plants. 
* $R$: the set of retailers.
* $p_i$: the production capacity of plant $i\in P$.
* $r_j$: the demand of retailer $j\in R$.
* $f_i$: operating plant $i$ incurs a fixed ramp-up cost $f_i$.
* $c_{ij}$: transhipping one unit from plant $i$ to retailer $j$ incurs transportation cost $c_{ij}$.

### Model components
#### Decisions:
We use variable $x_{ij}$ to decode the amount of goods from plant $i$ to retailer $j$.

We use variable $y_i$ to indicate whether plant $i$ is operation:
$$y_i = \begin{cases} 1,\quad &\text{if plant $i$ is in operation},\\
0,\quad &\text{otherwise}.
\end{cases}$$

Here, $x_{ij}\geq 0$ and $y_i\in\{0,1\}$ for $i\in P$ and $j\in R$.

#### Objective:
We want to minimize the total cost, which consists of the transportation cost and the ramp-up costs of plants.

$$\min\quad \sum_{i\in P} f_i y_i + \sum_{(i,j)\in A} c_{ij}x_{ij}$$

#### Supply constraints:
The production capacity of plant $i$ is $p_i$. 

$$\sum_{j: (i,j)\in A} x_{ij} \leq p_i y_i.$$

#### Demand constraints:
The demand of retailer $j$ is $r_j$.

$$\sum_{i: (i,j)\in A} x_{ij}\geq r_j.$$

### The integer programming model

\begin{align*}
\min\quad &\sum_{i\in P} f_i y_i + \sum_{(i,j)\in A} c_{ij}x_{ij}\\
\text{s.t.}\quad & \sum_{j: (i,j)\in A} x_{ij} \leq p_i y_i,\quad i\in P\\
&\sum_{i: (i,j)\in A} x_{ij}\geq r_j,\quad j\in R\\
&x_{ij}\geq 0,\ y_i\in\{0,1\},\quad i\in P,\ j\in R
\end{align*}

### Locations

This example considers two plants and nine retailer. 

The coordinates of each plant are provided in the following table.
The following table shows the coordinates of the plant locations and the fixed ramp-up cost of operating the plant.

| <i></i> | Coordinates |  Ramp-up costs |  Production capacities |
| --- | --- | --- | --- |
| Plant 1 | (0,1.5) | 5 | 60 |
| Plant 2 | (2.5,1.2) | 100 | 60 |
| Plant 3 | (1.7, 2.3) | 3 | 60 |
| Plant 4 | (0.7, 1.8) | 9 | 60 |

The coordinates of the retailers are provided in the following table.

| <i></i> | coordinates | Demands |
| --- | --- |  --- |
| Retailer 1 | (0,0) | 20 |
| Retailer 2 | (0,1) | 20 | 
| Retailer 3 | (0,2) | 20 | 
| Retailer 4 | (1,0) | 20 | 
| Retailer 5 | (1,1) | 20 |  
| Retailer 6 | (1,2) | 20 | 
| Retailer 7 | (2,0) | 20 | 
| Retailer 8 | (2,1) | 20 |   
| Retailer 9 | (2,2) | 20 | 

### Code to build the optimization model

In [4]:
from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

# Parameters
plants = [(0,1.5), (2.5,1.2), (1.7,2.3), (0.7, 1.8)]
retailers = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
ramp_up_cost = [5, 100, 3, 9]
production_capacities = [60, 60, 60, 60]
demands = [20, 20, 20, 20, 20, 20, 20, 20, 20]
cost_per_mile = 1

In [6]:
# This function determines the Euclidean distance between a plant and retailer sites.

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

# Compute key parameters of MIP model formulation

num_plants = len(plants)
num_retailers = len(retailers)
cartesian_prod = list(product(range(num_plants), range(num_retailers)))

# Compute transportation costs

transportation_cost = {(i,j): cost_per_mile*compute_distance(plants[i], retailers[j]) for i, j in cartesian_prod}

In [8]:
# MIP  model formulation

m = gp.Model('transportation')

plants_operation = m.addVars(num_plants, vtype=GRB.BINARY, name='Plant')
transport = m.addVars(cartesian_prod, vtype=GRB.CONTINUOUS, name='Transport')

m.addConstrs((gp.quicksum(transport[(i,j)] for j in range(num_retailers)) <= production_capacities[i] * plants_operation[i] for i in range(num_plants)), name='Production')
m.addConstrs((gp.quicksum(transport[(i,j)] for i in range(num_plants)) >= demands[j] for j in range(num_retailers)), name='Demand')

m.setObjective(plants_operation.prod(ramp_up_cost)+transport.prod(transportation_cost), GRB.MINIMIZE)

m.optimize()

Academic license - for non-commercial use only - expires 2024-03-04
Using license file /Users/dabeenlee/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 13 rows, 40 columns and 76 nonzeros
Model fingerprint: 0x8f388a9d
Variable types: 36 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [4e-01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Presolve time: 0.00s
Presolved: 13 rows, 40 columns, 76 nonzeros
Variable types: 36 continuous, 4 integer (4 binary)

Root relaxation: objective 2.093539e+02, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     209.3539487  209.35395  0.00%     -    0s

Explored 0 nodes (14 simplex iterati

## Analysis

In [12]:
# display optimal values of decision variables

for plant in plants_operation.keys():
    if (abs(plants_operation[plant].x) > 1e-6):
        print(f"\n Operate a plant at location {plant + 1}.")


 Operate a plant at location 1.

 Operate a plant at location 3.

 Operate a plant at location 4.


In [15]:
# Shipments from plants to retailers.

for plant, retailer in transport.keys():
    if (abs(transport[plant, retailer].x) > 1e-6):
        print(f"\n Retailer {retailer + 1} receives {transport[plant, retailer].x}   from Plant {plant + 1} .")


 Retailer 1 receives 20.0   from Plant 1 .

 Retailer 2 receives 20.0   from Plant 1 .

 Retailer 3 receives 20.0   from Plant 1 .

 Retailer 7 receives 20.0   from Plant 3 .

 Retailer 8 receives 20.0   from Plant 3 .

 Retailer 9 receives 20.0   from Plant 3 .

 Retailer 4 receives 20.0   from Plant 4 .

 Retailer 5 receives 20.0   from Plant 4 .

 Retailer 6 receives 20.0   from Plant 4 .
